# Advent of Code 2022

## Day 19: Not Enough Minerals

In [1]:
from anytree import Node, RenderTree, PreOrderIter, findall
from collections import namedtuple, deque
from functools import reduce
import re

In [2]:
Resources = namedtuple('Resources', ['ore', 'clay', 'obsidian', 'geode']) # Count of each resource
Robots    = namedtuple('Robots', ['ore', 'clay', 'obsidian', 'geode'])    # Count of each robot type
Costs     = namedtuple('Costs', ['ore', 'clay', 'obsidian', 'geode'])     # Cost (in Resources) of each robot type
Option    = namedtuple('Option', ['name', 'resources', 'robots'])         # Options for decision tree

In [3]:
def parse_input() -> list[Costs]:
    with open('Advent_19.txt', 'r') as f:
        lines = f.read().splitlines()
    
    costs_list = []
    
    for line in lines:
        line = line.strip()
        ore = int(re.search(r'ore robot costs (\d+) ore', line).group(1))
        clay = int(re.search(r'clay robot costs (\d+) ore', line).group(1))
        obsidian_match = re.search(r'obsidian robot costs (\d+) ore and (\d+) clay', line)
        obsidian_ore = int(obsidian_match.group(1))
        obsidian_clay = int(obsidian_match.group(2))
        geode_match = re.search(r'geode robot costs (\d+) ore and (\d+) obsidian', line)
        geode_ore = int(geode_match.group(1))
        geode_obsidian = int(geode_match.group(2))
        costs_list.append(Costs(Resources(ore, 0, 0, 0),
                                Resources(clay, 0, 0, 0),
                                Resources(obsidian_ore, obsidian_clay, 0, 0),
                                Resources(geode_ore, 0, geode_obsidian, 0)))
    return costs_list

In [4]:
def print_nodes(root) -> None:
    for _, _, n in RenderTree(root):
        print(f'{n.name:<12s}' + f'{str(n.resources):<50s}' + f'{str(n.robots):<50s}')

In [5]:
def build_tree(node, costs, time, stop_time, t_max) -> Node:
    if time > t_max:
        return node
    if time > stop_time:
        return node
    
    options = []
    
    # Robot building options
    if node.resources.ore >= costs.ore.ore:
        child_resources = Resources(node.resources.ore - costs.ore.ore + node.robots.ore,
                                    node.resources.clay + node.robots.clay,
                                    node.resources.obsidian + node.robots.obsidian,
                                    node.resources.geode + node.robots.geode)
        child_robots = Robots(node.robots.ore + 1,
                              node.robots.clay,
                              node.robots.obsidian,
                              node.robots.geode)
        options.append(Option(str(time) + '-ore', child_resources, child_robots))
    if node.resources.ore >= costs.clay.ore:
        child_resources = Resources(node.resources.ore - costs.clay.ore + node.robots.ore,
                                    node.resources.clay + node.robots.clay,
                                    node.resources.obsidian + node.robots.obsidian,
                                    node.resources.geode + node.robots.geode)
        child_robots = Robots(node.robots.ore,
                              node.robots.clay + 1,
                              node.robots.obsidian,
                              node.robots.geode)
        options.append(Option(str(time) + '-clay', child_resources, child_robots))
    if node.resources.ore >= costs.obsidian.ore and node.resources.clay >= costs.obsidian.clay:
        child_resources = Resources(node.resources.ore - costs.obsidian.ore + node.robots.ore,
                                    node.resources.clay - costs.obsidian.clay + node.robots.clay,
                                    node.resources.obsidian + node.robots.obsidian,
                                    node.resources.geode + node.robots.geode)
        child_robots = Robots(node.robots.ore,
                              node.robots.clay,
                              node.robots.obsidian + 1,
                              node.robots.geode)
        options.append(Option(str(time) + '-obsidian', child_resources, child_robots))
    if node.resources.ore >= costs.geode.ore and node.resources.obsidian >= costs.geode.obsidian:
        child_resources = Resources(node.resources.ore - costs.geode.ore + node.robots.ore,
                                    node.resources.clay + node.robots.clay,
                                    node.resources.obsidian - costs.geode.obsidian + node.robots.obsidian,
                                    node.resources.geode + node.robots.geode)
        child_robots = Robots(node.robots.ore,
                              node.robots.clay,
                              node.robots.obsidian,
                              node.robots.geode + 1)
        options.append(Option(str(time) + '-geode', child_resources, child_robots))

    # Option to build no robots
    child_resources = Resources(node.resources.ore + node.robots.ore,
                                node.resources.clay + node.robots.clay,
                                node.resources.obsidian + node.robots.obsidian,
                                node.resources.geode + node.robots.geode)
    child_robots = Robots(node.robots.ore, node.robots.clay, node.robots.obsidian, node.robots.geode)
    options.append(Option(str(time) + '-none', child_resources, child_robots))
    
    # Limit number of robots to max cost for the resource the robot produces.
    if node.robots.ore > max([c.ore for c in costs]):
        options = [opt for opt in options if opt.robots.ore == node.robots.ore]
    if node.robots.clay > max([c.clay for c in costs]):
        options = [opt for opt in options if opt.robots.clay == node.robots.clay]
    if node.robots.obsidian > max([c.obsidian for c in costs]):
        options = [opt for opt in options if opt.robots.obsidian == node.robots.obsidian]
        
    # If you can make a geode robot, do it.
    for opt in options:
        if opt.robots.geode > node.robots.geode:
            options = [opt]
            break    
            
    for opt in options:
        new_node = Node(opt.name, resources=opt.resources, robots=opt.robots)
        child = build_tree(new_node, costs, time=time+1, stop_time=stop_time, t_max=t_max)
        child.parent = node
        
    return node

In [6]:
def filter_duplicates(leaves) -> list[Node]:
    leaves = sorted(leaves, 
                    key=lambda n: (n.robots.geode, n.robots.obsidian, n.robots.clay, n.robots.ore,
                                   n.resources.geode, n.resources.obsidian, n.resources.clay, n.resources.ore), 
                    reverse=True)
    keepers = []
    prev_leaf = None
    for leaf in leaves:
        if prev_leaf == None:
            keepers.append(leaf)
            prev_leaf = leaf
            continue
        if leaf.robots == prev_leaf.robots and leaf.resources == prev_leaf.resources:
            continue    
        keepers.append(leaf)
        prev_leaf = leaf
        
    return keepers

In [14]:
def filter_at_ndx(leaves, ndx) -> list[Node]:
    keepers = []
    
    for i in range(len(leaves)-1):
        cur = leaves[i]
        nxt = leaves[i+1]
        cur_flat = [x for x in cur.resources]
        cur_flat.extend([x for x in cur.robots])
        nxt_flat = [x for x in nxt.resources]
        nxt_flat.extend([x for x in nxt.robots])
        
        # List of indices where current and next leaf differ
        diff_indices = [i for i, (c, n) in enumerate(zip(cur_flat, nxt_flat)) if c != n]
        
        # Leaves are sorted ascending, according to the index of interest, so if the current and next leaf
        # differ only at the index of interest, we hold out for the next one, which will be higher.
        if len(diff_indices) == 1 and diff_indices[0] == ndx:
            continue
            
        # We have reached a leaf that differs by more than just the index of interest.
        # Save the result of previous comparisons, which will be the one with the highest value
        # for the index of interest.
        keepers.append(cur)
    
    keepers.append(leaves[-1]) # Keep the last leaf, since there's no next leaf for comparison.    
    return keepers

In [8]:
def filter_inferior(leaves) -> list[Node]: 
    # Filter inferior resources.ore
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.clay, n.resources.obsidian, n.resources.geode,
                                   n.robots.ore, n.robots.clay, n.robots.obsidian, n.robots.geode,
                                   -n.resources.ore),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 0)
    
    # Filter inferior resources.clay
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.ore, n.resources.obsidian, n.resources.geode,
                                   n.robots.ore, n.robots.clay, n.robots.obsidian, n.robots.geode,
                                   -n.resources.clay),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 1)
    
    # Filter inferior resources.obsidian
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.ore, n.resources.clay, n.resources.geode,
                                   n.robots.ore, n.robots.clay, n.robots.obsidian, n.robots.geode,
                                   -n.resources.obsidian),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 2)    
    
    # Filter inferior robots.ore
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.ore, n.resources.clay, n.resources.obsidian, n.resources.geode,
                                   n.robots.clay, n.robots.obsidian, n.robots.geode,
                                   -n.robots.ore),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 4)
    
    # Filter inferior robots.clay
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.ore, n.resources.clay, n.resources.obsidian, n.resources.geode,
                                   n.robots.ore, n.robots.obsidian, n.robots.geode,
                                   -n.robots.clay),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 5)
    
    # Filter inferior robots.obsidian
    leaves = sorted(list(leaves), 
                    key=lambda n: (n.resources.ore, n.resources.clay, n.resources.obsidian, n.resources.geode,
                                   n.robots.ore, n.robots.clay, n.robots.geode,
                                   -n.robots.obsidian),
                    reverse=True)
    leaves = filter_at_ndx(leaves, 6)
    
    return leaves

In [9]:
def prune(root, leaves) -> list[Node]:
    # Filter out duplicate and inferior leaves
    leaves = filter_duplicates(leaves)
    leaves = filter_inferior(leaves)
    
    # Keep ancestors of the leaves we want to keep, and delete the rest.
    keepers = set()
    keepers.add(root)
    for leaf in leaves:
        n = leaf
        while n != root:
            keepers.add(n)
            n = n.parent
    for n in PreOrderIter(root, filter_=lambda n: n not in keepers):
        # We sever the reference from child to parent. We are assuming that anytree
        # will also sever the connection from parent to child, so memory can be freed.
        n.parent = None
        del n
        
    return leaves

In [10]:
def get_times(step, t_max) -> deque:
    times = deque()
    start_time = 1
    stop_time = 1+step
    times.append((start_time, stop_time))
    while stop_time < t_max:
        start_time += step+1
        stop_time += step+1
        times.append((start_time, stop_time))
    return times

In [11]:
def geodes_collected(costs_list, t_max, step=1) -> list[tuple]:
    geodes = []
    
    for blueprint, costs in enumerate(costs_list, 1):
        times = get_times(step, t_max)
        node = Node('root', resources=Resources(0,0,0,0), robots=Robots(1,0,0,0))
        start_time, stop_time = times.popleft()
        root = build_tree(node, costs, time=start_time, stop_time=stop_time, t_max=t_max)
        leaves = findall(root, filter_=lambda n: len(n.children) == 0)
        leaves = prune(root, leaves)
        
        while len(times) > 0:
            start_time, stop_time = times.popleft()
            for leaf in leaves:
                build_tree(leaf, costs, time=start_time, stop_time=stop_time, t_max=t_max)
            leaves = findall(root, filter_=lambda n: len(n.children) == 0)
            leaves = prune(root, leaves)
        
        geode_nodes = findall(root, filter_=lambda n: len(n.children) == 0)
        max_geode = 0
        max_node = None
        for n in geode_nodes:
            geode = n.resources.geode
            if geode > max_geode:
                max_geode = geode
                max_node = n

        #prune(root, [max_node])
        #print_nodes(root)
        #print((blueprint, max_geode))
        geodes.append((blueprint, max_geode))
    
    return geodes

### Part 1

In [12]:
costs = parse_input()
geodes = geodes_collected(costs, 24)
print(sum(b*g for b, g in geodes))

1147


### Part 2

In [13]:
geodes = geodes_collected(costs[:3], 32)
print(reduce(lambda x, y: x*y, [g for _, g in geodes], 1))

3080


### Comments
I spent a lot of time on this puzzle.  At first, I didn't read the problem correctly and developed a solution that allowed building as many robots as one can afford at each decision point.  It's important to read the problem carefully!

My first approach to pruning was to assume that the first branch to reach each new robot type would be the one that eventually leads to the optimal solution.  This is a bad assumption, but it gave the correct answer for the first test case, and this accidental success led me astray for a while.

My next attempt at pruning was to make a sorted list of the best nodes after several rounds of tree-building and keep the top portion of the list.  This heuristic leads to a fast result but not always a correct one.

Along the way, I went from building my own tree dataclass to using the anytree library.  I also got some helpful hints from Reddit, including a hint to avoid creating branches that build more robots than are needed.  (For instance, if the maximum cost of any obsidian or geode robot is 8 clay, then you will not benefit from building more than 8 clay robots.)

My final solution limits tree growth by:
- Not creating surplus robots
- Pruning duplicate nodes
- Pruning inferior nodes, i.e., nodes that match 7 attributes and fall short on the 8th.  (By "attribute," I mean the 4 resource types and the 4 robot types.)

Suspecting a tradeoff between the waste of building unnecessary branches and the cost of pruning them, I experimented with allowing the tree to build for several minutes before pruning it.  Even though my pruning approach relies any multiple sorting operations, I found that pruning every minute was faster than allowing the tree to grow.

My solution is still pretty slow.  To make it faster, I would probably need to examine series of nodes, instead of just comparing the nodes at one depth.

Stuff I learned from solving this puzzle:
- Named tuples, which resulted in more readable code, without the need to create custom classes
- Trees in general, and the Anytree library in particular.  Anytree was more robust than my homegrown approach.
- Recursion--still a somewhat confusing topic, but gradually becoming clearer
- Sorting with multiple criteria
- My first foray into type hints
- Other cool stuff: regex, zip() reduce(), queue